In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import gc
from tqdm import tqdm
from sklearn import preprocessing
from scipy.stats import entropy, kurtosis

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',100)
pd.set_option('display.max_info_rows',100)

In [6]:
import gc
gc.collect()

224

In [3]:
train_jet = pd.read_csv('E:/jet_complex_data/complex_train_R04_jet.csv')
train_particle = pd.read_feather('E:/jet_complex_data/complex_train_R04_particle.feather')
test_jet = pd.read_csv('E:/jet_complex_data/complex_test_R04_jet.csv')
test_particle = pd.read_feather('E:/jet_complex_data/complex_test_R04_particle.feather')

In [9]:
train_particle

,particle_category,particle_px,particle_py,particle_pz,particle_energy,particle_mass,jet_id
0,-211,-1.299730,0.368729,-0.057008,1.359400,0.13957,01291194f90c44c7bd79d9dbd50abd93
1,-2212,1.305420,0.418302,-0.426083,1.714930,0.93827,7df78235aa464796bbb386765d4784b5
2,22,-0.611265,0.111135,-0.225794,0.661044,0.00000,9aeaad70b98f4bf0890734137f88c7ac
3,211,3.778570,0.205956,-1.089380,3.940330,0.13957,4261ccf933b64ce6ba83c623e9a5bf06
4,22,0.461909,0.330447,-0.168640,0.592448,0.00000,e79cf654192e4cab8b83f50554fededc
...,...,...,...,...,...,...,...
24297347,22,-1.173750,0.414522,-0.616230,1.388980,0.00000,300f47688ccc47c2a59c7c674f8d5a46
24297348,211,-1.216210,0.033404,0.208287,1.242230,0.13957,853974ded875499597a0c160b718a5bd
24297349,22,2.128180,-0.497285,-1.109510,2.451010,0.00000,3104af57d9524ec9ae3fb3bfc326cd8a
24297350,211,-9.644300,4.915750,-1.419890,10.918500,0.13957,a99480901a29441d845efade2381f59a


In [17]:
def trans_data(df):
    df_out = df.set_index(['jet_id',df.groupby('jet_id').cumcount()+1]).unstack().sort_index(level=1, axis=1)
    df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)
    return(df_out.reset_index())

In [18]:
train_particle_trans = trans_data(train_particle)
train_particle_trans

,jet_id,particle_category_1,particle_energy_1,particle_mass_1,particle_px_1,particle_py_1,particle_pz_1,particle_category_2,particle_energy_2,particle_mass_2,particle_px_2,particle_py_2,particle_pz_2,particle_category_3,particle_energy_3,particle_mass_3,particle_px_3,particle_py_3,particle_pz_3,particle_category_4,particle_energy_4,particle_mass_4,particle_px_4,particle_py_4,particle_pz_4,particle_category_5,particle_energy_5,particle_mass_5,particle_px_5,particle_py_5,particle_pz_5,particle_category_6,particle_energy_6,particle_mass_6,particle_px_6,particle_py_6,particle_pz_6,particle_category_7,particle_energy_7,particle_mass_7,particle_px_7,particle_py_7,particle_pz_7,particle_category_8,particle_energy_8,particle_mass_8,particle_px_8,particle_py_8,particle_pz_8,particle_category_9,...,particle_py_101,particle_pz_101,particle_category_102,particle_energy_102,particle_mass_102,particle_px_102,particle_py_102,particle_pz_102,particle_category_103,particle_energy_103,particle_mass_103,particle_px_103,particle_py_103,particle_pz_103,particle_category_104,particle_energy_104,particle_mass_104,particle_px_104,particle_py_104,particle_pz_104,particle_category_105,particle_energy_105,particle_mass_105,particle_px_105,particle_py_105,particle_pz_105,particle_category_106,particle_energy_106,particle_mass_106,particle_px_106,particle_py_106,particle_pz_106,particle_category_107,particle_energy_107,particle_mass_107,particle_px_107,particle_py_107,particle_pz_107,particle_category_108,particle_energy_108,particle_mass_108,particle_px_108,particle_py_108,particle_pz_108,particle_category_109,particle_energy_109,particle_mass_109,particle_px_109,particle_py_109,particle_pz_109
0,000006d8f2c143e39d40961834db96e6,-211.0,2.909090,0.13957,-2.852690,-0.261343,-0.487022,22.0,2.018830,0.00000,-1.997200,0.105857,0.275090,22.0,0.153842,0.00000,-0.143739,-0.030189,-0.045775,22.0,0.935248,0.00000,-0.918017,0.104534,0.144931,211.0,17.093901,0.13957,-17.073400,-0.181165,0.805141,-211.0,1.116690,0.13957,-1.049710,-0.049222,-0.351022,22.0,1.770420,0.00000,-1.767700,-0.036279,0.091239,22.0,0.715917,0.00000,-0.711656,-0.037322,0.068479,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00005b43827d451fbf43b7a6b8c90099,-2112.0,11.596900,0.93957,-4.871760,-9.337350,-4.762990,130.0,5.357170,0.49761,-1.774390,-4.545590,-2.154250,211.0,4.268850,0.13957,-1.560010,-3.702500,-1.435780,211.0,6.390820,0.13957,-1.227880,-5.382080,-3.216940,-211.0,1.088360,0.13957,-0.337282,-0.879397,-0.527214,-211.0,0.594023,0.13957,-0.304849,-0.484107,-0.078045,-211.0,1.518000,0.13957,-0.631902,-1.162530,-0.730799,2212.0,8.326780,0.93827,-3.777230,-6.503020,-3.449370,-211.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00006de576aa41ca93b2549401436baf,-211.0,4.909550,0.13957,-4.884220,0.414131,0.238834,321.0,115.859001,0.49368,-115.853996,-0.861668,0.007566,22.0,33.277500,0.00000,-33.265701,-0.272818,-0.845967,321.0,9.737870,0.49368,-9.724540,0.005166,-0.124929,130.0,6.410900,0.49761,-6.366500,-0.554937,0.108283,22.0,4.233720,0.00000,-4.120150,0.309623,0.923508,-211.0,11.438100,0.13957,-11.436300,0.098086,0.110421,22.0,4.950190,0.00000,-4.781980,0.389701,1.218650,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000070820a23425aa2881cc5d5041735,211.0,1.383150,0.13957,-1.322880,0.090313,0.368065,22.0,1.340490,0.00000,-1.229850,0.513744,0.143054,211.0,2.831820,0.13957,-2.538250,1.198990,0.345603,-211.0,1.962470,0.13957,-1.915550,0.401423,-0.036586,22.0,12.797700,0.00000,-12.727400,1.328640,-0.168150,-211.0,1.5

In [ ]:
train_particle_trans.info(null_counts=True, ver)